In [ ]:
!pip install google.cloud --upgrade
!pip install replace
!pip install hana_ml

In [ ]:
import io
import time
from google.resumable_media.requests import download
from google.cloud import storage
from google.cloud.storage import blob
import pandas as pd
import replace 
import hana_ml.dataframe as dataframe
#Provide you connection details below either to ingest data into DWC Container or to HANA Cloud 
conn = dataframe.ConnectionContext(address = 'XXXXXXXXXXXXXXXXXXXXXXXXXXXXXX', #Provide your OpenSQL Host here
                                   port = 443, 
                                   user = 'XXXXXX', #Provide your OpenSQL container Username here 
                                   password = 'XXXXXXX', #Provide your OpenSQL contianer password here
                                   encrypt = 'true'
                                  )
storage_client = storage.Client.from_service_account_json('<Path to Json File>') #Provide the service account JSON File
buckets = list(storage_client.list_buckets())
t1=time.time()
blobs = storage_client.list_blobs('XXXXX')  #Provide the bucket name  
for blob in blobs:
        #print(blob.name)
        #with open(blob.name, "wb") as my_blob:

             #blob_data = blob.blob_client_instance.download_blob()
        blob_data = blob.download_as_string()
        blobread  = pd.read_csv(io.BytesIO(blob_data))
        key = blob.name
        keystr = key.replace(".csv","")
        df_remote = dataframe.create_dataframe_from_pandas(connection_context = conn, 
                                                   pandas_df = blobread, 
                                                   table_name = keystr,
                                                 #  schema = 'XXXX',   #Schema name is not needed for DWC Open SQL Container
                                                   force = True,
                                                   replace = True)
        t2=time.time()
        print(("It takes %s seconds to download "+blob.name) % (t2 - t1))